# 0. Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%%writefile setup.in
molecule h2o {
  O 
  H 1 0.96
  H 1 0.96 2 104.5
}

set basis cc-pVDZ
energy('scf')

compare_values(-76.0266327341067125, get_variable('SCF TOTAL ENERGY'), 6, 'SCF energy')

In [ ]:
!psi4 setup.in

## II. Limitations in Hartree–Fock theory

from lab by Eugene DePrince, FSU

The Hartree–Fock method is a very powerful and very important method in quantum chemistry, but it has a number of shortcomings. Here, you will explore two of these limitations.

##### A. Bond Breaking

Hartree–Fock theory cannot properly describe the breaking of chemical bonds, even in simple molecules like H$_2$! Compute a potential energy curve (PEC) for H$_2$ at the Hartree–Fock/6-31G and singles and doubles coupled cluster (CCSD)/6-31G levels of theory (for this system, CCSD will provide the exact result). Plot the energy as a function of bond length using the following bond lengths [Å]: 0.5, 0.7, 0.9, 1.1, 1.5, 2.0, 2.5, 3.0 .

In [ ]:
%%writefile hhbond.in
molecule h2 {
    H
    H 1 bond
}

set basis 6-31G

interatom_separations = []
hf_energies = []
ccsd_energies = []
for r in [0.5, 0.7, 0.9, 1.1, 1.5, 2.0, 2.5, 3.0]:
    h2.bond = r
    interatom_separations.append(r)

    e = energy('HF')
    hf_energies.append(e)
 
    clean()
    
    e = energy('ccsd')
    ccsd_energies.append(e)

print interatom_separations
print hf_energies
print ccsd_energies

In [ ]:
! psi4 hhbond.in

**Q.** Using your results, estimate the dissociation energy at the CCSD/6-31G level of theory (in kcal/mol). By how much does Hartree–Fock overestimate the dissociation energy, relative to CCSD?

In [ ]:
dist = [0.5, 0.7, 0.9, 1.1, 1.5, 2.0, 2.5, 3.0]
hfe = [-1.058027112142081, -1.1261243206686637, -1.1116878042209115, -1.0756881883148715, -0.997501528037092, -0.9162809010422871, -0.8569176468935037, -0.8156205218197542]
cce = [-1.077863898365415, -1.150156838995527, -1.1406024510970791, -1.1112698758321018, -1.0543474453294603, -1.0143102742983847, -1.00081314765349, -0.9974548300833993]

cc_dissoc = (max(cce) - min(cce)) * 627.5095
hf_dissoc = (max(hfe) - min(hfe)) * 627.5095

plt.plot(dist, hfe, 'r--o', dist, cce, 'g-.o')
plt.axis([0.45, 3.5, None, None])

plt.annotate(str(round(cc_dissoc, 2)), xy=(3.05, max(cce)), color='g')
plt.annotate('', xy=(3.0, min(cce)), xycoords='data',
                 xytext=(3.0, max(cce)), textcoords='data',
                 arrowprops=dict(arrowstyle="<->",
                                 connectionstyle="bar",
                                 ec="g",
                                 shrinkA=5, shrinkB=5))

plt.annotate(str(round(hf_dissoc, 2)), xy=(3.05, max(hfe)), color='r')
plt.annotate('', xy=(3.0, min(hfe)), xycoords='data',
                 xytext=(3.0, max(hfe)), textcoords='data',
                 arrowprops=dict(arrowstyle="<->",
                                 connectionstyle="bar",
                                 ec="r",
                                 shrinkA=5, shrinkB=5))

**Q.** Compute the non-bonded interaction energy for the FH dimer at the SCF/aug-cc-pVDZ level of theory in the linear configuration (F–H   F–H) using this procedure. Perform a geometry optimization of the F-H molecule. Record the final energy.

In [ ]:
%%writefile fhdim.in
molecule monomer {
    H
    F 1 1.0
}

molecule dimer {
    H 0.0 0.0 0.0
    F 0.0 0.0 1.0
    H 0.0 0.0 2.0
    F 0.0 0.0 3.0
}

set basis aug-cc-pvdz

e_fh = optimize('hf', molecule=monomer)
e_fhfh = optimize('hf', molecule=dimer)

e_int = e_fhfh - 2 * e_fh
print 'Monomer', e_fh
print 'Dimer', e_fhfh
print 'IE', e_int, e_int * 627.5095

In [ ]:
! psi4 fhdim.in

In [ ]:
! grep '~' fhdim.out

**Q.** What is the interaction energy? How does it compare with the interaction energy computed at the coupled-cluster through perturbative triple excitation [CCSD(T)]/aug-cc-pVDZ level of theory (`-3.846 [kcal/mol]`)?

Compute the interaction energy for the Argon dimer in the aug-cc-pVDZ basis. This time, do not optimize the Argon dimer geometry. Instread, assume an interatom separation of `3.999 [Å]`. How does your result compare to the CCSD(T)/aug-cc-pVDZ value of `-0.262 [kcal/mol]`. What does the sign of youre result say about the interaction (is it favorable/unfavorable)?

In [ ]:
%%writefile ardim.in

set basis aug-cc-pvdz

molecule ar {
    Ar
}

e_ar = energy('hf')

molecule arar {
    Ar
    Ar 1 3.999    
}

e_arar = energy('hf')

e_int = e_arar - 2 * e_ar
print 'Monomer', e_ar
print 'Dimer', e_arar
print 'IE', e_int, e_int * 627.5095
print 'CCSD(T) IE', -0.262

In [ ]:
! psi4 ardim.in

Below you will find the interaction energy (kcal/mol) of the argon dimer computed at the CCSD(T)/aug-cc-pVDZ level of theory. Compute and record the interaction energy for the ARgon dimer at the Hartree–Fock/aug-cc-pVDZ level of theory at these Ar-Ar distances [Å], and plot your results alongside the following CCSD(T) results.

In [ ]:
%%writefile ardimpec.in

set basis aug-cc-pvdz

molecule ar {
    Ar
}

e_ar = energy('hf')

molecule arar {
    Ar
    Ar 1 Rarar    
}

dist = [3.4, 3.6, 3.8, 4.0, 4.2, 4.4]
enes = []


for r in dist:
    arar.Rarar = r
    e_arar = energy('hf')
    
    ie = 627.5095 * (e_arar - 2 * e_ar)  # IE in kcal/mol
    enes.append(ie)

print dist
print enes

In [ ]:
! psi4 ardimpec.in

In [ ]:
dist = [3.4, 3.6, 3.8, 4.0, 4.2, 4.4]
hfe = [0.9284148251100534, 0.4336228746747894, 0.19190747942300562, 0.07595446731295193, 0.023432316097122794, 0.0018151033981354202]
cce = [0.136, -0.146, -0.244, -0.262, -0.245, -0.214]

In [ ]:
plt.plot(dist, hfe, 'r--o', dist, cce, 'g-.o')
plt.axhline(0, color='grey')

Does Hartree-Fock predict a minimum in the IE? Why or why not? What types of intermolecular interactions would you expect to dominate in the FH dimer and the Ar dimer? Based on your results, what can you say about the ability of the Hartree–Fock method to describe these types of interactions?

Does Hartree-Fock predict a minimum in the IE? Why or why not? What types of intermolecular interactions would you expect to dominate in the FH dimer and the Ar dimer? Based on your results, what can you say about the ability of the Hartree–Fock method to describe these types of interactions